<a href="https://colab.research.google.com/github/spyrohiro/spyrohiro/blob/main/Extra_curric1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile


zip_ref = zipfile.ZipFile("/content/drive/MyDrive/archive.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# How many images in each folder
import os

# Walk through 10 percent data directory and list number of files
for root, dir, files in os.walk("PetImages"):
  print(f"There are {len(dir)} directories and {len(files)} images in '{root}'.")

There are 2 directories and 0 images in 'PetImages'.
There are 0 directories and 12501 images in 'PetImages/Dog'.
There are 0 directories and 12501 images in 'PetImages/Cat'.


In [ ]:
# Split the data into train and test using imagedatagenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator


IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
train_data_dir = "/content/PetImages/"

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_data_1 = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training') # set as training data

test_data_1 = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation') # set as validation data


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Create TensorBoard callback (functionized because we need to create a new one for each model)
import datetime 

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Savign Tensorboard logfiles to: {log_dir}")
  return tensorboard_callback

# Creating a feature extraction model using tensorflow hub

In [ ]:
# Pre-trained model links:

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import dependencies

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Create model building function that uses transfer learnin

def create_model(model_url, num_classes=2):
  """
  takes tensorflow hub url and creates a keras sequential model with it.
  args:
  model_url (str): a tensorflor hub feature extraction url.
  num_classes (int): number of output neurons in the output layers,
  should be equal to number of target classes, default: 2.

  returns:
  An uncompiled keras sequential model with model_url as feature extractor
  layer and dense output layer with num_classes output neurns.

  """ 
  # Download the pretrained model and save it as a keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))
  
  # Create out own model

  model = tf.keras.Sequential([ 
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model

# Creating Effientnet TensorFlow Hub feature extraction model

In [ ]:
# Create effientnet model

efficientnet_model = create_model(efficientnet_url,
                                  num_classes=train_data_1.num_classes)

In [ ]:
efficientnet_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 1280)              4049564   
_________________________________________________________________
output_layer (Dense)         (None, 2)                 2562      
Total params: 4,052,126
Trainable params: 2,562
Non-trainable params: 4,049,564
_________________________________________________________________


In [ ]:
efficientnet_model.compile(loss="sparse_categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

In [ ]:
from PIL import Image

In [ ]:
efficientnet_history = efficientnet_model.fit(train_data_1,
                                              epochs=5,
                                              steps_per_epoch=len(train_data_1),
                                              validation_data=test_data_1,
                                              validation_steps=len(test_data_1),
                                              callbacks=[create_tensorboard_callback("Tensorflow_hub", "effcientnet")])


Savign Tensorboard logfiles to: Tensorflow_hub/effcientnet/20210831-154638
Epoch 1/5
 88/625 [===>..........................] - ETA: 4:19 - loss: 0.0258 - accuracy: 0.9901

UnknownError: ignored